# 07_compute_ILC

compute interlobe correlation

In [ ]:
import numpy as np
from os.path import join as pjoin
from os.path import isdir
import os
import matplotlib.pyplot as plt
from matplotlib import cm, colors
import mne_bids
import mne
from mne_bids import write_raw_bids, BIDSPath
from scipy import stats
import re
from scipy import signal
import pandas as pd
from scipy import signal, fftpack

## define functions

In [ ]:
def compute_intercorr(sub_idx, run, bids_root):
    
    sub_path = BIDSPath(subject=sub_idx, run=int(run), task='movie', session='movie', processing='preproc', root=bids_root)
    raw_sub = mne_bids.read_raw_bids(sub_path)
    ref_path = BIDSPath(subject='02', run=int('01'), task='movie', session='movie', processing='preproc', root=bids_root)
    raw_ref = mne_bids.read_raw_bids(ref_path)
    # A reference is necessary for detecting bad channel, I simply use the original raw data for the conveinence.
    
    ch_name_picks = mne.pick_channels_regexp(raw_sub.ch_names, regexp='M[LRZ]...-4503')
    type_picks = mne.pick_types(raw_sub.info, meg=True)
    sub_picks= np.intersect1d(ch_name_picks, type_picks)
    ch_name_picks_ref = mne.pick_channels_regexp(raw_ref.ch_names, regexp='M[LRZ]...-4503')
    type_picks_ref = mne.pick_types(raw_ref.info, meg=True)
    ref_picks= np.intersect1d(ch_name_picks_ref, type_picks_ref)
    if len(sub_picks) == len(ref_picks):
        if (sub_picks == ref_picks).all():
            picks = sub_picks
        else:
            picks = np.intersect1d(sub_picks, ref_picks)
        bad_idx = []
    else:
        picks = sub_picks if len(sub_picks) > len(ref_picks) else ref_picks
        bad_picks = np.union1d(np.setdiff1d(sub_picks, ref_picks), np.setdiff1d(ref_picks, sub_picks))
    
        bad_idx = []
        for chn in bad_picks:
            bad_idx.append(np.where(picks == chn)[0][0])

    sub_data = raw_sub.get_data(picks=picks)
    sub_envlope = np.abs(signal.hilbert(sub_data))
    
    if len(bad_idx) != 0:
        for idx in bad_idx:
            sub_envlope[idx,:] = 0
    
    #downsampling
    events_sub = mne.events_from_annotations(raw_sub)
    sample_sub = events_sub[0][1:-1:25,0]
    sub_dsamp = sub_envlope.take(sample_sub, axis=1)
    
    intra_lobe_corr = np.zeros((len(picks), len(picks)))

    for i in np.arange(len(picks)):
        sub_data1 = sub_dsamp.take(i, axis=0)
        
        for j in np.arange(len(picks)):
            sub_data2 = sub_dsamp.take(j, axis=0)
        
            r, p = stats.pearsonr(sub_data1, sub_data2)
            
            intra_lobe_corr[i][j] = intra_lobe_corr[j][i] = r
    
    ch_name = []
    for idx in picks:
        ch_name.append(raw_sub.ch_names[idx])
    labels = ['LC', 'LF', 'LO', 'LP', 'LT', 'RC', 'RF', 'RO', 'RP', 'RT']
    ch_labels = dict()
    for label in labels:
        reg = re.compile('M'+label+'.*')
        tmp_chs = list(filter(reg.match, ch_name))
        ch_labels[label] = [ch_name.index(x) for x in tmp_chs]
    
    mean_lobe_corr = np.zeros((len(labels), len(labels)))
    
    for i, (roi_x, ch_idx_x) in enumerate(ch_labels.items()):
        for j, (roi_y, ch_idx_y) in enumerate(ch_labels.items()):
            tmp = intra_lobe_corr.take(ch_idx_x, axis=0)
            roi_val = tmp.take(ch_idx_y, axis=1)
            mean_lobe_corr[i][j] = np.mean(roi_val)
            
    return mean_lobe_corr


## define variables

In [ ]:
bids_root = '/nfs/e5/studyforrest/forrest_movie_meg/preproc_data'
labels = ['LC', 'LF', 'LO', 'LP', 'LT', 'RC', 'RF', 'RO', 'RP', 'RT']
sub_list = ['{0:0>2d}'.format(sub) for sub in np.arange(1,12)]
run_list = ['{0:0>2d}'.format(run) for run in np.arange(1,9)]
data_pth = '/nfs/s2/userhome/daiyuxuan/workingdir/MEG-paper/output_data'

## compute Interlobe correlation

In [ ]:
interlobe_corr = {}
for sub in sub_list:
    interlobe_corr[sub] = []
    if sub == '01':
        run_ls = run_list + ['09']
    else:
        run_ls = run_list
    for run in run_ls:
        mean_lobe_corr = compute_intercorr(sub, run, bids_root)
        interlobe_corr[sub].append(mean_lobe_corr)
        
# save ILC data
for sub in sub_list[1:]:
    interlobe_corr[sub].append(np.nan)
df = pd.DataFrame(interlobe_corr, columns=sub_list, index=run_list+['09'])
df.to_pickle(pjoin(data_pth, 'interlobe_correlation.pickle'))